In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
import json

In [2]:
df = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')

In [4]:
# Mengubah seluruh text kedalam bentuk lowercase
df['sentence'] = df['sentence'].str.lower()
 
# Menghilangkan stopwords
stop_word = set(stopwords.words('english'))
 
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
 
# Melakukan split dataset
sentence = df['sentence'].values
label = df['label'].values
 
sentence_train, sentence_test, label_train, label_test = train_test_split(sentence, label, test_size=0.2, shuffle=False)
 
# Membuat tokenisasi
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Filter untuk menghilangkan symbols
 
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)
 
tokenizer.fit_on_texts(sentence_train)
 
# Menyimpan word_index kedalam sebuah file json
word_index = tokenizer.word_index
 
with open('word_index.json', 'w') as fp:
    json.dump(word_index, fp)
 
# Membuat sequences dan melakukan padding
train_sekuens = tokenizer.texts_to_sequences(sentence_train)
test_sekuens = tokenizer.texts_to_sequences(sentence_test)
 
train_padded = pad_sequences(train_sekuens,
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

In [5]:
# Membuat model
model = tf.keras.Sequential([
    Embedding(2000, 20, input_length=20),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
 
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
 
# Train model
num_epochs = 30
history = model.fit(train_padded, label_train,
                    epochs=num_epochs,
                    validation_data=(test_padded, label_test),
                    verbose=1)

Epoch 1/30
25/25 [==============================] - 2s 30ms/step - loss: 0.6921 - accuracy: 0.5288 - val_loss: 0.7111 - val_accuracy: 0.2400
Epoch 2/30
25/25 [==============================] - 1s 38ms/step - loss: 0.6866 - accuracy: 0.5650 - val_loss: 0.7415 - val_accuracy: 0.2400
Epoch 3/30
25/25 [==============================] - 2s 65ms/step - loss: 0.6803 - accuracy: 0.5650 - val_loss: 0.7688 - val_accuracy: 0.2400
Epoch 4/30
25/25 [==============================] - 0s 17ms/step - loss: 0.6691 - accuracy: 0.5650 - val_loss: 0.7632 - val_accuracy: 0.2400
Epoch 5/30
25/25 [==============================] - 0s 10ms/step - loss: 0.6329 - accuracy: 0.6187 - val_loss: 0.7435 - val_accuracy: 0.2900
Epoch 6/30
25/25 [==============================] - 0s 6ms/step - loss: 0.5416 - accuracy: 0.7900 - val_loss: 0.7064 - val_accuracy: 0.4450
Epoch 7/30
25/25 [==============================] - 0s 7ms/step - loss: 0.3939 - accuracy: 0.8925 - val_loss: 0.6096 - val_accuracy: 0.6600
Epoch 8/30
25/2

In [6]:
model.save("model.h5")

In [8]:
!pip install tensorflowjs


  Using cached tensorflowjs-3.20.0-py3-none-any.whl (81 kB)
  Using cached jax-0.3.19.tar.gz (1.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached importlib_resources-5.9.0-py3-none-any.whl (33 kB)
  Using cached flax-0.6.0-py3-none-any.whl (180 kB)
  Using cached protobuf-3.19.5-cp310-cp310-win_amd64.whl (895 kB)
  Using cached optax-0.1.3-py3-none-any.whl (145 kB)
  Using cached msgpack-1.0.4-cp310-cp310-win_amd64.whl (61 kB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached rich-11.2.0-py3-none-any.whl (217 kB)
  Using cached etils-0.8.0-py3-none-any.whl (127 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached zipp-3.8.1-py3-none-any.whl (5.6 kB)
  Using cached chex-0.1.5-py3-none-any.whl (85 kB)
  Using cached optax-0.1.2-py3-none-any.whl (140 kB)
  Using cached optax-0.1.1-py3-none-any.whl (136 kB)
  Using cached optax-0.1.0-py3-none-any.whl (126 kB)
  Usi

In [9]:
# Convert model.h5 to model
!tensorflowjs_converter --input_format=keras model.h5 tfjs_model

2022-09-28 16:09:30.213386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-09-28 16:09:30.214532: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
